2.ミクロカノニカル(NVE一定)分子動力学シミュレーション

In [14]:
% matplotlib inline
import math
import numpy as np            
import matplotlib.pyplot as plt

In [15]:
N=108
FREE=3.0
VX=np.zeros(N)
VY=np.zeros(N)
VZ=np.zeros(N)
FX=np.zeros(N)
FY=np.zeros(N)
FZ=np.zeros(N)
seed=2017
#np.random.seed(seed)
np.random.rand()

0.6691868059272222

In [16]:
#   READ IN INITIAL PARAMETERS 
#" MOLECULAR DYNAMICS OF LENNARD-JONES ATOMS
#" ENTER NUMBER OF STEPS
NSTEP=10000
#ENTER NUMBER OF STEPS BETWEEN OUTPUT LINES
IPRINT=1000
#ENTER THE FOLLOWING IN LENNARD-JONES UNITS
#ENTER THE DENSITY
DENS=0.50
#ENTER THE POTENTIAL CUTOFF DISTANCE
RCUT=0.20
#ENTER THE CORE RADIUS TO AVOID OVERLAP
RMIN=2.0
# ENTER THE TIME STEP
DT=100
L = pow(N / DENS, 1.0 / 3.0)
if RCUT > 0.5*L:
        RCUT = 0.5*L
print(" NUMBER OF ATOMS      =  %10d\n" % N);
print(" NUMBER OF STEPS        =  %10d\n" % NSTEP);
print(" OUTPUT FREQUENCY    =  %10d\n"% IPRINT);
print(" POTENTIAL CUTOFF     =  %10.4f\n" % RCUT);
print(" DENSITY                         =  %10.4f\n" % DENS);
print(" TIME STEP                      =  %10.6f\n" % DT);

 NUMBER OF ATOMS      =         108

 NUMBER OF STEPS        =       10000

 OUTPUT FREQUENCY    =        1000

 POTENTIAL CUTOFF     =      0.2000

 DENSITY                         =      0.5000

 TIME STEP                      =  100.000000



In [17]:
#RANDOM INITIAL CONFIGURATION
RX = np.random.rand(N) - 0.5
RY = np.random.rand(N) - 0.5
RZ = np.random.rand(N) - 0.5
for i in range(N):
    for j in range(i):
        DX = (RX[i] - RX[j])
        DY = (RY[i] - RY[j])
        DZ = (RZ[i] - RZ[j])
        DX -= np.floor(DX + 0.5)
        DY -= np.floor(DY + 0.5)
        DZ -= np.floor(DZ + 0.5)
        DR = np.sqrt(DX*DX + DY*DY + DZ*DZ)
        if DR*L < RMIN:
            --i
            break

In [18]:
#CALCULATE LONG-RANGE CORRECTIONS 
SR3 = pow(1.0 / RCUT, 3.0)
SR9 = SR3 * SR3 * SR3
VLRC12 =  8.0 * np.pi *DENS * N * SR9 / 9.0
VLRC6  = -8.0 * np.pi * DENS *N * SR3 / 3.0
VLRC = VLRC12 + VLRC6
WLRC12 = 12.0 * VLRC12
WLRC6  =  6.0 * VLRC6
WLRC = WLRC12 + WLRC6

#ZERO ACCUMULATORS 
ACV  = 0.0
ACK  = 0.0
ACE  = 0.0
ACEC = 0.0
ACP  = 0.0
ACT  = 0.0
ACVSQ  = 0.0
ACKSQ  = 0.0
ACESQ  = 0.0
ACECSQ = 0.0
ACPSQ  = 0.0
ACTSQ  = 0.0
FLV  = 0.0
FLK  = 0.0
FLE  = 0.0
FLEC = 0.0
FLP  = 0.0
FLT  = 0.0

In [19]:
def FORCE(pL, pRCUT, pV, pVC, pW, RX, RY, RZ, VX, VY, VZ, FX, FY, FZ):
    
    RCUTSQ = pow(pRCUT / pL, 2.0)
    SIGSQ  = pow(pL, -2.0)
    for I in range(N):
        FX[I] = 0.0
        FY[I] = 0.0
        FZ[I] = 0.0
    
    pV = 0.0
    pW = 0.0
    NCUT = 0
#CONDUCT OUTER LOOP OVER ATOMS 
    for I in range(N-1):
        RXI = RX[I]
        RYI = RY[I]
        RZI = RZ[I]
        FXI = FX[I]
        FYI = FY[I]
        FZI = FZ[I]
#CONDUCT INNER LOOP OVER ATOMS
        for J in range(I+1, N):
            RXIJ = RXI - RX[J]
            RYIJ = RYI - RY[J]
            RZIJ = RZI - RZ[J]
#MINIMUM IMAGE THE PAIR SEPARATIONS
            RXIJ -= np.floor(RXIJ + 0.5)
            RYIJ -= np.floor(RYIJ + 0.5)
            RZIJ -= np.floor(RZIJ + 0.5)
            RIJSQ = RXIJ*RXIJ + RYIJ*RYIJ + RZIJ*RZIJ
            if RIJSQ < RCUTSQ :
                SR2   = SIGSQ / RIJSQ
                SR6   = SR2 * SR2 * SR2
                SR12  = SR6 * SR6
                VIJ   = 4.0 * (SR12 - SR6)
                WIJ   = 24.0 * (2.0 * SR12 - SR6)
                pV   += VIJ
                pW   += WIJ
                FIJ   = WIJ / RIJSQ
                FXIJ  = FIJ * RXIJ
                FYIJ  = FIJ * RYIJ
                FZIJ  = FIJ * RZIJ
                FXI   += FXIJ
                FYI   += FYIJ
                FZI   += FZIJ
                FX[J] -= FXIJ
                FY[J] -= FYIJ
                FZ[J] -= FZIJ
                ++NCUT
#END OF INNER LOOP 
        FX[I] = FXI
        FY[I] = FYI
        FZ[I] = FZI

#CONVERT FORCE INTO LJ UNIT 
        for I in range(N):
            FX[I] /= pL
            FY[I] /= pL
            FZ[I] /= pL
#CALCULATE POTENTIAL SHIFT  */
        SR2  = SIGSQ / RCUTSQ
        SR6  = SR2 * SR2 * SR2
        SR12 = SR6 * SR6
        VIJ  = 4.0 * (SR12 -SR6)
        pVC = pV - NCUT * VIJ

In [20]:
def KINET(pK, VX, VY, VZ):
#COMPUTES KINETIC ENERGY
    pK = 0.0;
    for I in range(N):
        pK += VX[I]*VX[I] + VY[I]*VY[I] + VZ[I]*VZ[I]
    pK *= 0.5;

In [21]:
def MOVEA(pDT, pL, RX, RY, RZ, VX, VY, VZ, FX, FY, FZ):
    DT2 = pDT / 2.0
    DTSQ2 = pDT * DT2
    for I in range(N):
        RX[I] += (pDT*VX[I] + DTSQ2*FX[I]) / pL
        RY[I] += (pDT*VY[I] + DTSQ2*FY[I]) / pL
        RZ[I] += (pDT*VZ[I] + DTSQ2*FZ[I]) / pL
        VX[I] += DT2 * FX[I]
        VY[I] += DT2 * FY[I]
        VZ[I] += DT2 * FZ[I]

In [22]:
def MOVEB(pDT, pK, RX, RY, RZ, VX, VY, VZ, FX, FY, FZ):
#SECOND PART OF VELOCITY VERLET ALGORITHM 
    DT2 = pDT / 2.0
    pK = 0.0
    for I in range(N):
        VX[I] += DT2 * FX[I]
        VY[I] += DT2 * FY[I]
        VZ[I] += DT2 * FZ[I]
        pK += VX[I]*VX[I] + VY[I]*VY[I] + VZ[I]*VZ[I]
    pK *= 0.5

In [23]:
#CALCULATE INITIAL VALUES  
V=0
VC=0
W=0
K=0
FORCE(L, RCUT, V, VC, W,RX, RY, RZ, VX, VY, VZ, FX, FY, FZ)
KINET(K, VX, VY, VZ)
KN = K / N
TEMP = 2.0 * KN / FREE
VS = (V + VLRC) / N
WS = (W + WLRC) / N
PS = DENS * TEMP + (W + WLRC) / (3.0 * L*L*L)
print(" INITIAL V/N       =  %10.4f\n" % VS);
print(" INITIAL W/N       =  %10.4f\n" % WS);
print(" INITIAL P         =  %10.4f\n" %PS);

 INITIAL V/N       =  2726553.3575

 INITIAL W/N       =  32721781.8822

 INITIAL P         =  5453630.3137



In [24]:
#/*******************************************************************/
#/*  MAIN LOOP BEGINS                                               */
#/*******************************************************************/
print("\n**** START OF DYNAMICS ****\n\n");
print("    TIMESTEP     ENERGY    CUTENERGY    KINETIC    POTENTIAL    PRESSURE  TEMPERATURE \n");
for STEP in range(NSTEP+ 1):
#/*  VELOCITY VERLET ALGORITHM         */
#/*  Update positions r(t) -> r(t+dt)  */
#/*  and Velocities v(t) -> v(t+dt/2)  */
    MOVEA(DT, L, RX, RY, RZ, VX, VY, VZ, FX, FY, FZ)
#/*  Calculate force f(t+dt)  */
    FORCE(L, RCUT, V, VC, W,RX, RY, RZ, VX, VY, VZ, FX, FY, FZ)
#/*  Update velocities v(t+dt/2) -> v(t+dt) by using f(t+dt)  */
    MOVEB(DT, K, RX, RY, RZ, VX, VY, VZ, FX, FY, FZ)
#/*  INCLUDE LONG-RANGE CORRECTIONS  */
    V += VLRC
    W += WLRC
#/*  CALCULATE INSTANTANEOUS VALUES  */
    E    = K + V
    EC   = K + VC
    VN   = V  / float(N)
    KN   = K  / float(N)
    EN   = E  / float(N)
    ECN  = EC / float(N)
    TEMP = 2.0 * KN / FREE
    PRES = DENS * TEMP + W / (3.0 * L*L*L)
#/*  INCREMENT ACCUMULATORS  */
    ACE  += EN
    ACEC += ECN
    ACK  += KN
    ACV  += VN
    ACP  += PRES
    ACESQ  += EN * EN
    ACECSQ += ECN * ECN
    ACKSQ  += KN * KN
    ACVSQ  += VN * VN
    ACPSQ  += PRES * PRES
#/*  PERFORM PERIODIC OPERATIONS    */
#/*  WRITE OUT RUNTIME INFORMATION  */
    if STEP%IPRINT == 0:
        print("  %8d  %10.4f  %10.4f  %10.4f  %10.4f  %10.4f  %10.4f\n" %(STEP, EN, ECN, KN, VN, PRES, TEMP))
print("\n**** END OF DYNAMICS ****\n\n");
#/*******************************************************************/
#/*  MAIN LOOP ENDS                                                 */
#/*******************************************************************/


**** START OF DYNAMICS ****


    TIMESTEP     ENERGY    CUTENERGY    KINETIC    POTENTIAL    PRESSURE  TEMPERATURE 

         0  2726553.3575      0.0000      0.0000  2726553.3575  5453630.3137      0.0000

      1000  2729279910.8231      0.0000      0.0000  2729279910.8231  5459083944.0203      0.0000

      2000  5455833268.2885      0.0000      0.0000  5455833268.2885  10912714257.7273      0.0000

      3000  8182386625.7545      0.0000      0.0000  8182386625.7545  16366344571.4336      0.0000

      4000  10908939983.2201      0.0000      0.0000  10908939983.2201  21819974885.1393      0.0000

      5000  13635493340.6850      0.0000      0.0000  13635493340.6850  27273605198.8449      0.0000

      6000  16362046698.1498      0.0000      0.0000  16362046698.1498  32727235512.5506      0.0000

      7000  19088600055.6146      0.0000      0.0000  19088600055.6146  38180865826.2562      0.0000

      8000  21815153413.0806      0.0000      0.0000  21815153413.0806  43634496139.

In [25]:
#/*  OUTPUT RUN AVERAGES  */
NORM = float(NSTEP)
AVE  = ACE  / NORM
AVEC = ACEC / NORM
AVK  = ACK  / NORM
AVV  = ACV  / NORM
AVP  = ACP  / NORM
ACESQ  = (ACESQ  / NORM) - AVE * AVE
ACECSQ = (ACECSQ / NORM) - AVEC* AVEC
ACKSQ  = (ACKSQ  / NORM) - AVK * AVK
ACVSQ  = (ACVSQ  / NORM) - AVV * AVV
ACPSQ  = (ACPSQ  / NORM) - AVP * AVP

if ACESQ  > 0.0:
    FLE  = math.sqrt(ACESQ)
if ACECSQ > 0.0:
    FLEC = math.sqrt(ACECSQ)
if ACKSQ  > 0.0:
    FLK  = math.sqrt(ACKSQ)
if ACVSQ  > 0.0:
    FLV  = math.sqrt(ACVSQ)
if ACPSQ  > 0.0:
    FLP  = math.sqrt(ACPSQ)
AVT = AVK * 2.0 / FREE
FLT = FLK * 2.0 / FREE

print("  AVERAGES   %10.5f  %10.5f  %10.5f  %10.5f  %10.5f  %10.5f\n" %(AVE, AVEC, AVK, AVV, AVP, AVT))
print("   FLUCTS    %10.5f  %10.5f  %10.5f  %10.5f  %10.5f  %10.5f\n"%(FLE, FLEC, FLK, FLV, FLP, FLT) )

  AVERAGES   13636856890.01905     0.00000     0.00000  13636856890.01905  27276332559.36439     0.00000

   FLUCTS    7870881062.76119     0.00000     0.00000  7870881062.76119  15743273624.89202     0.00000

